In [ ]:
from __future__ import print_function
! pip install turicreate

In [ ]:
import turicreate
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
import time
from scipy.sparse import csr_matrix
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
%matplotlib inline

In [ ]:
wiki = turicreate.SFrame('/content/drive/MyDrive/people_wiki.sframe')

In [ ]:
wiki['tfidf'] = turicreate.text_analytics.tf_idf(wiki['text'])

**En iyi kelimeleri kelime sayılarına göre TF-IDF ile karşılaştırın**

In [ ]:
selected_one = wiki[wiki['name'] == 'Elton John']
selected_one

URI,name,text,tfidf
<http://dbpedia.org/resource/Elton_John> ...,Elton John,sir elton hercules johncbe born reginald ken ...,"{'movements':5.030658019760364, ..."


In [ ]:
selected_one['word_count'] = turicreate.text_analytics.count_words(selected_one['text'])
selected_one

Columns:
	URI	str
	name	str
	text	str
	tfidf	dict
	word_count	dict

Rows: 1

Data:
+-------------------------------+------------+-------------------------------+
|              URI              |    name    |              text             |
+-------------------------------+------------+-------------------------------+
| <http://dbpedia.org/resour... | Elton John | sir elton hercules john cb... |
+-------------------------------+------------+-------------------------------+
+-------------------------------+-------------------------------+
|             tfidf             |           word_count          |
+-------------------------------+-------------------------------+
| {'movements': 5.0306580197... | {'movements': 1.0, 'social... |
+-------------------------------+-------------------------------+
[1 rows x 5 columns]

**en yüksek kelime sayısına sahip 3 kelime**

In [ ]:
sorted(selected_one['word_count'][0].items(), key = lambda item:item[1], reverse=True)[0:3]

[('the', 27.0), ('in', 18.0), ('and', 15.0)]

**en yüksek TF-IDF'ye sahip 3 kelime**

In [ ]:
wiki['tfidf'] = turicreate.text_analytics.tf_idf(wiki['text'])
wiki.head()

URI,name,text,tfidf
<http://dbpedia.org/resource/Digby_Morrell> ...,Digby Morrell,digby morrell born 10october 1979 is a former ...,"{'melbourne':3.8914310119380633, ..."
<http://dbpedia.org/resource/Alfred_J._Lewy> ...,Alfred J. Lewy,alfred j lewy aka sandylewy graduated from ...,"{'time':1.3253342074200498, ..."
<http://dbpedia.org/resource/Harpdog_Brown> ...,Harpdog Brown,harpdog brown is a singerand harmonica player who ...,"{'society':2.4448047262085693, ..."
<http://dbpedia.org/resource/Franz_Rottensteiner> ...,Franz Rottensteiner,franz rottensteiner bornin waidmannsfeld lower ...,"{'kurdlawitzpreis':10.986495389225194, ..."
<http://dbpedia.org/resource/G-Enka> ...,G-Enka,henry krvits born 30december 1974 in tallinn ...,"{'curtis':5.299520032885375, ..."
<http://dbpedia.org/resource/Sam_Henderson> ...,Sam Henderson,sam henderson bornoctober 18 1969 is an ...,"{'asses':9.600201028105303, 's ..."
<http://dbpedia.org/resource/Aaron_LaCrate> ...,Aaron LaCrate,aaron lacrate is anamerican music producer ...,"{'streamz':10.986495389225194, ..."
<http://dbpedia.org/resource/Trevor_Ferguson> ...,Trevor Ferguson,trevor ferguson aka johnfarrow born 11 november ...,"{'concordia':6.250296940830698, ..."
<http://dbpedia.org/resource/Grant_Nelson> ...,Grant Nelson,grant nelson born 27april 1971 in london ...,"{'heavies':8.907053847545358, 'n ..."
<http://dbpedia.org/resource/Cathy_Caruth> ...,Cathy Caruth,cathy caruth born 1955 isfrank h t rhodes ...,"{'2002':1.8753125887822302, ..."


In [ ]:
selected_one = wiki[wiki['name'] == 'Elton John']
selected_one

URI,name,text,tfidf
<http://dbpedia.org/resource/Elton_John> ...,Elton John,sir elton hercules johncbe born reginald ken ...,"{'movements':5.030658019760364, ..."


**Measuring distance**

In [ ]:
elton = wiki[wiki['name'] == 'Elton John']
beckham = wiki[wiki['name'] == 'Victoria Beckham']
mcCartney = wiki[wiki['name'] == 'Paul McCartney']

**'Elton John' ve 'Victoria Beckham' arasındaki cos mesafesi**

In [ ]:
turicreate.distances.cosine(elton['tfidf'][0],beckham['tfidf'][0])

0.9567006376655429

**'Elton John' ve 'Paul McCartney' arasındaki cos mesafesi**

In [ ]:
turicreate.distances.cosine(elton['tfidf'][0],mcCartney['tfidf'][0])

0.8250310029221779

**Nearest Neighbor model**

In [ ]:
tfidf_model = turicreate.nearest_neighbors.create(wiki,features=['tfidf'],label='name', distance='cosine')

Starting brute force nearest neighbors model training.

Validating distance components.

Initializing model data.

Initializing distances.

Done.

In [ ]:
wiki['word_count'] = turicreate.text_analytics.count_words(wiki['text'])
word_count_model = turicreate.nearest_neighbors.create(wiki,features=['word_count'],label='name', distance='cosine')

Starting brute force nearest neighbors model training.

Validating distance components.

Initializing model data.

Initializing distances.

Done.

In [ ]:
elton = wiki[wiki['name'] == 'Elton John']
beckham = wiki[wiki['name'] == 'Victoria Beckham']
mcCartney = wiki[wiki['name'] == 'Paul McCartney']

**"Elton John" konulu habere kendisinden başka kelime kullanılarak en çok benzeyen makale**

In [ ]:
word_count_model.query(elton)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 14.809ms     |

| Done         |         | 100         | 463.148ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Elton John,2.220446049250313e-16,1
0,Cliff Richard,0.16142415258967036,2
0,Sandro Petrone,0.16822542751041114,3
0,Rod Stewart,0.16832716558706107,4
0,Malachi O'Doherty,0.177315545978884,5


**TF-IDF kullanılarak 'Elton John' konulu makaleye kendisinden başka en çok benzeyen makale**

In [ ]:
tfidf_model.query(elton)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 19.083ms     |

| Done         |         | 100         | 522.482ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Elton John,-2.220446049250313e-16,1
0,Rod Stewart,0.7172196678927374,2
0,George Michael,0.7476009989692848,3
0,Sting (musician),0.7476719544306141,4
0,Phil Collins,0.7511932487904706,5


**"Victoria Beckham" konulu makaleye kendisinden başka en çok benzeyen makale**

In [ ]:
word_count_model.query(beckham)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 17.894ms     |

| Done         |         | 100         | 455.276ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Victoria Beckham,-2.220446049250313e-16,1
0,Mary Fitzgerald (artist),0.20730703611504997,2
0,Adrienne Corri,0.21450978278754795,3
0,Beverly Jane Fry,0.21746646874079278,4
0,Raman Mundair,0.21769547499150488,5


**TF-IDF kullanılarak 'Victoria Beckham' konulu makaleye kendisinden başka en çok benzeyen makale**

In [ ]:
tfidf_model.query(beckham)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 19.367ms     |

| Done         |         | 100         | 552.935ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Victoria Beckham,1.1102230246251565e-16,1
0,David Beckham,0.5481696102632145,2
0,Stephen Dow Beckham,0.7849867068283364,3
0,Mel B,0.8095855234085036,4
0,Caroline Rush,0.81982642291868,5
